# Training

This notebook demonstrates the process of training an ensemble learning model using a provided CSV file. It showcases data preprocessing, model training, evaluation, and saving the trained model. The ensemble method (hard voting, soft voting, or stacking) can be selected based on the user's choice.


## Importing Necessary Libraries

First, we import all the necessary libraries and modules needed for this script. This includes libraries for handling warnings, data manipulation, machine learning, and the custom Ensemble module containing ensemble learning methods.

In [13]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, 
    recall_score, 
    precision_score, 
    f1_score
)
from src import Ensemble, Bayes, LSTM, BERT
import math

## Utility Functions

### Function to Read CSV File

The `read_csv_file` function reads the CSV file and returns a pandas DataFrame. If the file is not found, the script will exit with an error message.

In [2]:
def read_csv_file(filename: str) -> pd.DataFrame:
    try:
        data = pd.read_csv(filename, lineterminator='\n', usecols=range(2))
        print("CSV file read successfully!")
        return data
    except FileNotFoundError:
        print("ERROR: File not found")
        exit(1)

# Demonstrate reading a CSV file (use a sample or mock filename)
dataset = read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [12]:
dataset['label'].value_counts(ascending=True)

label
0    14115
1    14346
Name: count, dtype: int64

### Function to Seed Random Number Generators

To ensure reproducibility, the `seed_random_number_generators` function seeds the random number generators for PyTorch and NumPy.

In [3]:
def seed_random_number_generators(seed=0):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    print("Random number generators seeded.")

# Seed the random number generators
seed_random_number_generators()

Random number generators seeded.


### Function for Train-Test Split

The `get_train_test_split` function splits the dataset into training and testing sets with an 80-20 split ratio and returns them.

In [4]:
def shuffle_data_frame(data_frame):
    text = list(data_frame['text'])
    label = list(data_frame['label'])

    assert(len(text) == len(label))

    indices = list(range(len(label)))

    # Make a random number generator that will shuffle list of indices
    # It is seeded to be reproducible
    random_number_generator = np.random.default_rng(seed=0)
    random_number_generator.shuffle(indices)

    shuffled_text = []
    shuffled_labels = []

    # Iterate through the list of indices and add the original data
    # from those shuffled indices
    for index in indices:
        shuffled_text.append(text[index])
        shuffled_labels.append(label[index])

    return pd.DataFrame({
        'text': shuffled_text,
        'label': shuffled_labels,
    })


def get_train_test_split(data_frame: pd.DataFrame, test_size: float):
    """
    Makes a stratified train test split.
    This aims to preserve the distribution between classes.
    """
    if not (1 >= test_size >= 0):
        print('ERROR: test_size must be between 0 and 1')
        return

    data_frame = shuffle_data_frame(data_frame)

    data_frame_length = len(data_frame)
    train_size = 1 - test_size

    nonhate_rows = data_frame[data_frame['label'] == 0] 
    nonhate_row_length = len(nonhate_rows)

    nonhate_row_train_size = math.ceil(nonhate_row_length * train_size)

    nonhate_row_train = nonhate_rows[0:nonhate_row_train_size]
    nonhate_row_test = nonhate_rows[nonhate_row_train_size:nonhate_row_length]

    assert(len(nonhate_row_train) + len(nonhate_row_test) == nonhate_row_length)

    hate_rows = data_frame[data_frame['label'] == 1] 
    hate_row_length = len(hate_rows)

    hate_row_train_size = math.ceil(hate_row_length * train_size)

    hate_row_train = hate_rows[0:hate_row_train_size]
    hate_row_test = hate_rows[hate_row_train_size:hate_row_length]

    assert(len(hate_row_train) + len(hate_row_test) == hate_row_length)

    combined_train = pd.concat([nonhate_row_train, hate_row_train])
    combined_test = pd.concat([nonhate_row_test, hate_row_test])

    shuffled_train = shuffle_data_frame(combined_train)
    shuffled_test = shuffle_data_frame(combined_test)

    return (
        shuffled_train['text'],
        shuffled_test['text'],
        shuffled_train['label'],
        shuffled_test['label'],
    )

X_train, X_test, y_train, y_test = get_train_test_split(dataset, 0.2)

## Train Data

In [5]:
pd.DataFrame(X_train)

,text
0,[USERNAME] Palangga ka man sang mga taga Baco...
1,Who dafuq is Jose Montemayor Jr.???
2,Di na nakakatuwa yung mukha ni Mar Roxas sa TV...
3,national elections. | via[USERNAME]
4,"Binay will be staring in a movie called ""The D..."
...,...
22764,"""Kala ko wala andito pala si Marcos.""*pertaini..."
22765,sie ~ [USERNAME]Marcos Magnanakaw Marcos Dikta...
22766,If Mar is BatMarBinay is Bane-ay.
22767,to my moots im sorry in not sorry for flooding...


In [6]:
y_train_dataframe = pd.DataFrame(y_train, columns=['label'])
y_train_dataframe

,label
0,0
1,0
2,1
3,0
4,1
...,...
22764,0
22765,1
22766,1
22767,1


In [12]:
y_train_dataframe.value_counts(ascending=True)

label
0        11292
1        11477
Name: count, dtype: int64

## Test Data

In [8]:
pd.DataFrame(X_test)

,text
0,Bakit trending ang Only Binay?
1,Mare @ Cebu [USERNAME][USERNAME] Marcos Never ...
2,Kahit anong gawin ko bakit di ko ma appreciate...
3,Oras na para tayo'y bumoto ng taong mag tataas...
4,VP[USERNAME]is currently in Zamboanga Sibugay ...
...,...
5687,[USERNAME] Laban LeniAngat Buhay LahatLeni Kiko
5688,Nagconcede ka man Maimarwala ka prinnagdala ka...
5689,Did You Know that former Philippine secretary ...
5690,Bakit nakakairita commercial ni Mar Roxas?


In [9]:
y_test_dataframe = pd.DataFrame(y_test, columns=['label'])
y_test_dataframe

,label
0,0
1,1
2,1
3,0
4,0
...,...
5687,0
5688,1
5689,0
5690,1


In [10]:
y_test_dataframe.value_counts(ascending=True)

label
0        2823
1        2869
Name: count, dtype: int64

## Grid Search

### Bernoulli Naive Bayes

In [22]:
cross_validator = StratifiedKFold(n_splits=5)

nb_grid_search = GridSearchCV(
  estimator=Bayes.BayesPipeline,
  param_grid={
    'bayes__alpha': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    'bayes__force_alpha': [False, True],
  },
  scoring=['accuracy', 'precision', 'recall', 'f1'],
  refit='accuracy',
  cv=cross_validator,
)

nb_grid_search.fit(X_train, y_train)

/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:633: UserWarnin

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['akin',
                                                                    'aking',
                                                                    'ako',
                                                                    'alin',
                                                                    'am',
                                                                    'amin',
                                                                    'aming',
                                                                    'ang',
                                                                    'ano',
                                                                    'anumang',
                                                                    'apat',
                                                                    'at',
                                                                    'atin',
                                                                    'ating',
                                                                    'ay',
                                                                    'bababa',
                                                                    'bago',
                                                                    'bakit',
                                                                    'bawat',
                                                                    'bilang',
                                                                    'dahil',
                                                                    'dalawa',
                                                                    'dapat',
                                                                    'din',
                                                                    'dito',
                                                                    'doon',
                                                                    'gagawin',
                                                                    'gayunman',
                                                                    'ginagawa',
                                                                    'ginawa', ...])),
                                       ('bayes', BernoulliNB())]),
             param_grid={'bayes__alpha': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
                         'bayes__force_alpha': [False, True]},
             refit='accuracy',
             scoring=['accuracy', 'precision', 'recall', 'f1'])

In [23]:
nb_grid_search.cv_results_

{'mean_fit_time': array([0.51526742, 0.48221383, 0.56515198, 0.54316058, 0.54379282,
        0.54369125, 0.56222711, 0.54059415, 0.53951893, 0.54264879,
        0.48067026, 0.49868836]),
 'std_fit_time': array([0.03347867, 0.00735723, 0.06698537, 0.02326728, 0.00953409,
        0.00571984, 0.0316702 , 0.00416228, 0.00741439, 0.01542321,
        0.0023286 , 0.00687449]),
 'mean_score_time': array([0.1149817 , 0.11269097, 0.14183369, 0.12423892, 0.12758465,
        0.12492566, 0.12986035, 0.1293364 , 0.12791314, 0.12741041,
        0.11581264, 0.11706367]),
 'std_score_time': array([0.00405036, 0.00363129, 0.03198623, 0.0051025 , 0.00264587,
        0.0022377 , 0.004564  , 0.0022927 , 0.00440311, 0.00170536,
        0.00221504, 0.00355707]),
 'param_bayes__alpha': masked_array(data=[0.0, 0.0, 0.2, 0.2, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8, 1.0,
                    1.0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
  

In [24]:
nb_grid_search.best_params_

{'bayes__alpha': 0.6, 'bayes__force_alpha': False}

## Ensemble Training Function

The `train_ensemble` function initializes and trains the ensemble model using the provided training data. It takes the training features and labels as input, along with the ensemble model instance, and returns the trained model.

In [ ]:
def train_ensemble(X_train: list, y_train: list, ensemble):
    seed_random_number_generators()  # Ensure reproducibility
    ensemble.fit(X_train, y_train)
    print("Ensemble model trained.")
    return ensemble

## Prediction and Evaluation Function

The `get_prediction_results` function uses the trained ensemble model to make predictions on the test set and then evaluates these predictions by calculating the accuracy, recall, precision, and F1-score. It returns these metrics for further analysis.

In [ ]:
def get_prediction_results(X_test: list, y_test: list, ensemble):
    with torch.inference_mode():
        y_pred = ensemble.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        print(f"Accuracy: {accuracy}\nRecall: {recall}\nPrecision: {precision}\nF1-score: {f1}")
        return accuracy, recall, precision, f1

## Save Model Function

The `save_trained_model` function saves the trained ensemble model to disk using the joblib library. This allows for the model to be reloaded and used for predictions without the need for retraining.

In [ ]:
def save_trained_model(ensemble, filename="Ensemble"):
    import joblib
    joblib.dump(ensemble, f'{filename}.pkl', compress=True)
    print(f"Ensemble model saved to {filename}.pkl")

## Main Execution Workflow

This cell combines all the previous steps to execute the workflow. It includes reading the dataset, splitting it into training and testing sets, selecting the ensemble method, training the model, evaluating its performance, and saving the trained model. Replace 'your_dataset.csv' with the path to your dataset and choose an appropriate ensemble method.

In [ ]:
FILENAME = 'datasets/datasetall.csv'
ENSEMBLE_METHOD = 'hard'

# Read data and prepare train-test split
data_frame = read_csv_file(FILENAME)
X_train, X_test, y_train, y_test = get_train_test_split(data_frame)

# Initialize and train the ensemble
ensemble_methods = {
    'hard': Ensemble.HardVotingEnsemble(),
    'soft': Ensemble.SoftVotingEnsemble(),
    'stacking': Ensemble.StackingEnsemble(),
}
ensemble = train_ensemble(X_train, y_train, ensemble_methods[ENSEMBLE_METHOD])

# Evaluate the trained ensemble and display results
accuracy, recall, precision, f1 = get_prediction_results(X_test, y_test, ensemble)

# Save the trained model
save_trained_model(ensemble, f'ensemble-{ENSEMBLE_METHOD}')

## Results

To better visualize the evaluation results, this cell creates a pandas DataFrame to display the accuracy, recall, precision, and F1-score in a tabular format.

In [ ]:
import pandas as pd
results_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Recall', 'Precision', 'F1-Score'],
    'Value': [accuracy, recall, precision, f1]
})
results_df